In [1]:
import sys
sys.path.append('../../')

from bokehComponents.Dashboard import Dashboard
from bokehComponents.BokehComponents import BokehControl,BufferedQueryInterface,BokehTimeseriesGraphic

from bokeh.io import output_notebook,output_file
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.models.widgets import Panel, Tabs #, DataTable, DateFormatter, TableColumn, Tabs, 
from bokeh.models import ColumnDataSource
from bokeh.layouts import layout
from bokeh.plotting import figure,show,output_file
from bokeh.models import Div

from datetime import datetime,timedelta
import pandas_datareader as pdr


In [2]:
from bokeh.plotting import figure, output_file, show



# Bokeh Components
In this notebook we will outline how to use one of the most basic Bokeh Components, the button. Other notebooks show more exciting features, and are each designed to be independent. Bokeh Componets is an open source library that allows python developers to create interactive we applications with Bokeh more quickly. It was born out of frustration fighting with analytics libraries, and out of a need to share our analytics process internally. We are happy to share these time saving object oriented libraries with the world.

### Status: In Development
Note, the library is in active development, and virtually not method name or class name, or parameter is stable. Having said that, we have developed a suite of unit tests which we we commit to passing, at least internally.

### The Dashboard
Every Bokeh component has be be embedded into an application. To do this we extend from the basic dashboard application, and add in our button 

In [3]:
class BlankDashboard(Dashboard):
    def getLayout(self):
        l = layout([Div(text="Example <b>HTML</b>", width=200, height=100)],sizing_mode='fixed')         
        return l
d = BlankDashboard()
d.showDashboard()

Loading BokehJS ...

## Creating Data using the BufferedQueryInterface
With all Bokeh components, it can be useful to abstrat the data interface from the rest of the application. The BufferedQueryInterface allows a developer the ability to create highly functional / auto-updating data soruces that can support seamless integreation with several classes.

In [31]:
class ExampleComplexQuery(BufferedQueryInterface):
    '''
    An example of a data query that supports complex queries.
    '''
    
    #### Interface below -- Should be overriden by data sources
    #
    #
    #
    #
    def load_data_buffer(self):
        start_init=self._settings['start_date']
        end_init=self._settings['end_date']     

        # Grab data
        start=start_init
        end=end_init 

        data=pdr.get_data_yahoo('AAPL',start,end)
        data_sorted=data.sort_index(axis=0,ascending=True)
        date_list=list(data_sorted.index)

        # Save the data elements
        open_list=list(data_sorted['Open'])
        close_list=list(data_sorted['Close'])
        date_time=[datetime.strptime(str(d),'%Y-%m-%d %H:%M:%S').date() for d in date_list]
        
        self.data = {
            'open_list':open_list,
            'close_list':close_list,
            'date_time':date_time,
            }

In [32]:
import pandas as pd
query = ExampleQuery({'start_date':datetime(2017,4,26),'end_date':datetime(2017,7,26)})
data = query.QueryData()
df = pd.DataFrame(data)
display(df.head())

,open_list,close_list,date_time
0,144.470001,143.679993,2017-04-26
1,143.919998,143.789993,2017-04-27
2,144.089996,143.649994,2017-04-28
3,145.100006,146.580002,2017-05-01
4,147.539993,147.509995,2017-05-02


### Using the default BokehTimeseriesGraphic
It is possible use the graphic timeseries out of the box

In [33]:
class GraphDashboard(Dashboard):
    def getLayout(self):
        settings = {'title':'Analytics Plot',
                          'width':600,
                          'height':300,
                          'query':ExampleQuery({'start_date':datetime(2017,4,26),
                                               'end_date':datetime(2017,5,26)})}

        settings['data_defs'] = [
                    {'y':'open_list','x':'date_time','key':'open','label':'Open','color':'green' },
                    {'y':'close_list','x':'date_time','key':'close','label':'Close','color':'yellow' }                    
                    ]
        
        self.g = BokehTimeseriesGraphic(settings)
        l = layout([self.g.getBokehComponent()],sizing_mode='fixed')         
        return l    
    
d = GraphDashboard()
d.showDashboard()

Loading BokehJS ...